In [1]:
import os
import pandas as pd
from plotnine import *

In [2]:
# Sets the working directory.
os.chdir("C:\\Users\\fshen\OneDrive\\Documents\\AI Precision Nutrition Bootcamp\\T2D")

### Subjects

In [3]:
# Create data frame from "Subjects.txt" using [tab] as separator, top row as header
subjects = pd.read_csv("Subjects.txt", sep = "\t")
subjects.head()

,SubjectID,Study,Race,Sex,Age,BMI,SSPG,IR_IS_classification
0,ZIS22OE,HMP,C,F,NaN,NaN,NaN,Unknown
1,ZJBOZ2X,Exercise,C,F,65.30,19.82,NaN,Unknown
2,ZJOSZHK,HMP,C,M,41.43,19.42,NaN,Unknown
3,ZJTKAE3,HMP,C,F,58.65,31.24,162.0,IR
4,ZJXC41N,"HMP, Exercise",B,F,49.69,28.24,75.0,IS


In [4]:
# Select only subjects with known insulin sensitivity and known BMI and known race
subjects_IR_IS_known = subjects[(subjects["IR_IS_classification"] != "Unknown") 
                                 & (subjects["BMI"].notna())
                                 & (subjects["Race"] != "unknown")]
subjects_IR_IS_known.describe()

,Age,BMI,SSPG
count,64.000000,64.000000,64.000000
mean,56.191250,28.846719,149.830625
std,7.402171,3.788701,65.370466
min,38.800000,21.470000,40.000000
25%,50.960000,25.970000,91.875000
50%,56.430000,28.745000,156.935000
75%,61.942500,31.312500,213.547500
max,69.000000,38.900000,276.000000


In [5]:
# Factorize sex, insulin sensitivity, race, then reorganize to only show important data

subjects_IR_IS_known["Sex"] = subjects_IR_IS_known["Sex"].astype("category")

IR_IS_cats = pd.CategoricalDtype(categories = ["IS", "IR"], ordered = False)
subjects_IR_IS_known["IR_IS_classification"] = subjects_IR_IS_known["IR_IS_classification"].astype(IR_IS_cats)

race_cats = pd.CategoricalDtype(categories = ["C", "A", "B", "H"], ordered = False)
subjects_IR_IS_known["Race"] = subjects_IR_IS_known["Race"].astype(race_cats)

subjects_IR_IS_known = subjects_IR_IS_known.rename({"IR_IS_classification": "IR_IS"}, axis = 1)
subjects_IR_IS_known = subjects_IR_IS_known.drop(columns = ["Study"])
subjects_IR_IS_known.shape

C:\Users\fshen\AppData\Local\Temp\ipykernel_2756\1549532638.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\fshen\AppData\Local\Temp\ipykernel_2756\1549532638.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\fshen\AppData\Local\Temp\ipykernel_2756\1549532638.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

(64, 7)

In [6]:
subjects_IR_IS_known["IR_IS"].value_counts()

IR_IS
IR    34
IS    30
Name: count, dtype: int64

### Visits

In [7]:
# Create data frame from "Visits.txt" using [tab] as separator, top row as header
visits = pd.read_csv("Visits.txt", sep = "\t")
visits.shape

(1416, 8)

In [8]:
# Select only the visits from subjects listed in subjects_IR_IS_known
visits_known = visits[visits["SubjectID"].isin(subjects_IR_IS_known["SubjectID"])]
visits_known.shape

(1098, 8)

### Clinical Tests

In [9]:
# Create data frame from "clinical_tests.txt" using [tab] as separator, top row as header
clinicals = pd.read_csv("HMP/Clinical/clinical_tests.txt", sep = "\t")
clinicals.shape

(969, 57)

In [10]:
# Rename SubjectID to SubjectID_2 (reserve SubjectID for another column)
clinicals = clinicals.rename({"SubjectID": "SubjectID_2"}, axis = 1)

# New SubjectID column using VisitID (split using '-')
clinicals["SubjectID"] = clinicals["VisitID"].str.split(pat = '-')
clinicals.head()

,VisitID,A1C,AG,ALB,ALCRU,ALKP,ALT,AST,BASO,BASOAB,...,TP,UALB,UALBCR,WBC,SubjectID_2,CL1,CL2,CL3,CL4,SubjectID
0,ZOZOW1T-1013,6.0,8,4.0,NaN,96.0,48,22,0.6,0.04,...,6.3,NaN,NaN,6.0,69-001,D7,NaN,Infection_Late,Infection,"[ZOZOW1T, 1013]"
1,ZOZOW1T-1015,5.9,8,4.2,NaN,103.0,77,120,0.9,0.04,...,6.5,NaN,NaN,5.0,69-001,D30,NaN,Infection_Recovery_Late,Infection_L,"[ZOZOW1T, 1015]"
2,ZOZOW1T-1021,6.3,NaN,NaN,173.5,NaN,NaN,NaN,1.0,0.09,...,NaN,7,<30,8.9,69-001,D1,NaN,Infection_Early,Infection,"[ZOZOW1T, 1021]"
3,ZOZOW1T-1022,6.1,7,4.2,278.2,69.0,40,27,0.5,0.05,...,6.6,16,<30,10.8,69-001,D3,NaN,Infection_Middle,Infection,"[ZOZOW1T, 1022]"
4,ZOZOW1T-1023,6.3,13,4.2,412.8,66.0,53,31,0.6,0.04,...,6.7,18,<30,7.0,69-001,D15,NaN,Infection_Recovery_Early,Infection_L,"[ZOZOW1T, 1023]"


In [11]:
# Only keep rows with corresponding data in subjects_IR_IS_known and visits_known
clinicals_known = clinicals[clinicals["VisitID"].isin(visits_known["VisitID"])]
clinicals_known.describe()

,A1C,ALB,ALKP,BASO,BASOAB,BUN,CA,CHOL,CHOLHDL,CL,...,NA.,NEUT,NEUTAB,NHDL,PLT,RBC,RDW,TGL,TP,WBC
count,695.000000,702.000000,702.000000,697.000000,697.000000,702.000000,702.000000,700.000000,700.000000,702.000000,...,702.000000,698.000000,698.000000,700.000000,698.000000,698.000000,698.000000,700.000000,702.000000,698.000000
mean,5.642158,3.979060,81.641026,0.704304,0.039440,16.390313,8.970085,196.371429,3.468571,103.679487,...,139.535613,56.190258,3.220401,135.994286,221.269341,4.679742,13.802006,103.964286,7.039174,5.684241
std,0.433262,0.282999,25.135738,0.349110,0.021284,5.260551,0.356860,33.812227,1.046069,2.511851,...,2.494676,8.565239,1.123932,35.161552,51.088936,0.448061,1.092309,58.065353,0.403750,1.550714
min,4.700000,3.000000,38.000000,0.100000,0.000000,8.000000,7.800000,107.000000,1.400000,96.000000,...,131.000000,34.300000,0.900000,37.000000,117.000000,3.200000,11.700000,22.000000,5.700000,2.200000
25%,5.300000,3.800000,63.000000,0.500000,0.030000,13.000000,8.700000,175.000000,2.700000,102.000000,...,138.000000,50.825000,2.490000,114.000000,187.000000,4.330000,13.100000,64.750000,6.800000,4.600000
50%,5.600000,4.000000,77.500000,0.600000,0.040000,15.000000,9.000000,195.000000,3.350000,104.000000,...,140.000000,56.500000,3.090000,133.000000,215.000000,4.705000,13.600000,89.500000,7.100000,5.400000
75%,5.900000,4.200000,95.000000,0.800000,0.050000,18.750000,9.200000,216.000000,4.100000,105.000000,...,141.000000,62.175000,3.797500,156.000000,247.000000,5.000000,14.300000,129.000000,7.300000,6.600000
max,8.000000,4.900000,198.000000,5.000000,0.280000,40.000000,10.300000,332.000000,9.500000,110.000000,...,148.000000,95.500000,12.940000,297.000000,457.000000,6.380000,20.500000,407.000000,8.200000,15.300000


In [12]:
# Count values, sort by alphabetical order (Ant -> antibiotic, Imz -> immunization)
# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6666404/
clinicals_known["CL4"].value_counts().sort_index()

CL4
Allergy            2
Ant               14
Ant_L              4
Colonoscopy        3
Colonoscopy_L      1
Fiber             31
Healthy          406
Imz               69
Imz_L             23
Infection         66
Infection_L       36
Post-Travel        2
Stress             2
Stress_L           1
Weight-gain       22
Weight-loss       23
Name: count, dtype: int64

### Exploratory Analysis

In [13]:
plot_list = list()

plot_list.append(ggplot(subjects_IR_IS_known, aes(fill = "IR_IS"))
                 + geom_histogram(aes(x = "Age"), bins = 5, position = "dodge") 
                 + ylab("Count") 
                 + scale_fill_manual(values = ("skyblue", "orange")) 
                 + theme_bw())

plot_list.append(ggplot(subjects_IR_IS_known, aes(fill = "IR_IS")) 
                 + geom_histogram(aes(x = "SSPG"), bins = 5, position = "dodge")
                 + ylab("Count") 
                 + scale_fill_manual(values = ("skyblue", "orange")) 
                 + theme_bw())

plot_list.append(ggplot(subjects_IR_IS_known, aes(fill = "IR_IS")) 
                 + geom_histogram(aes(x = "BMI"), bins = 5, position = "dodge") 
                 + ylab("Count") 
                 + scale_fill_manual(values = ("skyblue", "orange")) 
                 + theme_bw())

plot_list.append(ggplot(subjects_IR_IS_known, aes(fill = "IR_IS"))
                 + geom_bar(aes(x = "Sex")) 
                 + scale_fill_manual(values = ("skyblue", "orange")) 
                 + theme_bw())

plot_list.append(ggplot(subjects_IR_IS_known, aes(fill = "IR_IS")) 
                 + geom_bar(aes(x = "Race")) 
                 + scale_fill_manual(values = ("skyblue", "orange")) 
                 + theme_bw())

for i in range(len(plot_list)):
    plot_list[i].save(filename = "plot" + str(i) + ".png", dpi = 300)


c:\Users\fshen\AppData\Local\Programs\Python\Python39\lib\site-packages\plotnine\ggplot.py:606: PlotnineWarning: Saving 6.4 x 4.8 in image.
c:\Users\fshen\AppData\Local\Programs\Python\Python39\lib\site-packages\plotnine\ggplot.py:607: PlotnineWarning: Filename: plot0.png
c:\Users\fshen\AppData\Local\Programs\Python\Python39\lib\site-packages\plotnine\ggplot.py:606: PlotnineWarning: Saving 6.4 x 4.8 in image.
c:\Users\fshen\AppData\Local\Programs\Python\Python39\lib\site-packages\plotnine\ggplot.py:607: PlotnineWarning: Filename: plot1.png
c:\Users\fshen\AppData\Local\Programs\Python\Python39\lib\site-packages\plotnine\ggplot.py:606: PlotnineWarning: Saving 6.4 x 4.8 in image.
c:\Users\fshen\AppData\Local\Programs\Python\Python39\lib\site-packages\plotnine\ggplot.py:607: PlotnineWarning: Filename: plot2.png
c:\Users\fshen\AppData\Local\Programs\Python\Python39\lib\site-packages\plotnine\ggplot.py:606: PlotnineWarning: Saving 6.4 x 4.8 in image.
c:\Users\fshen\AppData\Local\Programs\Pyt